## Exploring the Data

In [4]:
import json

# Path to the uncompressed users.json file
users_file = 'data/users.json'

# Function to load line-delimited JSON
def load_ndjson(filepath):
    data = []
    with open(filepath, 'r') as f:
        for line in f:
            if line.strip():  # Skip empty lines
                data.append(json.loads(line))
    return data

# Load the users data
users_data = load_ndjson(users_file)

# Check the structure of the first few records
print(users_data[:1])

[{'_id': {'$oid': '5ff1e194b6a9d73a3a9f1052'}, 'active': True, 'createdDate': {'$date': 1609687444800}, 'lastLogin': {'$date': 1609687537858}, 'role': 'consumer', 'signUpSource': 'Email', 'state': 'WI'}]


In [6]:
import pandas as pd

# Convert the loaded JSON data into a pandas DataFrame
users_df = pd.DataFrame(users_data)

# Inspect the first few rows
users_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [8]:
import pandas as pd

# Load the JSON files into pandas DataFrames
users_df = pd.read_json('data/users.json', lines=True)
receipts_df = pd.read_json('data/receipts.json', lines=True)
brands_df = pd.read_json('data/brands.json', lines=True)

In [9]:
# Check basic structure and data types
print(users_df.info())
print(receipts_df.info())
print(brands_df.info())

# Check first few rows for initial inspection
print(users_df.head())
print(receipts_df.head())
print(brands_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              11

## Checking for missing values

In [10]:
# Check for missing values in each column
print("Missing values in Users DataFrame:")
print(users_df.isnull().sum())

print("Missing values in Receipts DataFrame:")
print(receipts_df.isnull().sum())

print("Missing values in Brands DataFrame:")
print(brands_df.isnull().sum())


Missing values in Users DataFrame:
_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64
Missing values in Receipts DataFrame:
_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64
Missing values in Brands DataFrame:
_id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
dtype: int64


## Checking for Duplicates

In [11]:
# Check for duplicate records in key fields like 'user_id', 'receipt_id', etc.
print("Duplicate records in Users DataFrame (by user_id):")
print(users_df.duplicated(subset=['_id']).sum())

print("Duplicate records in Receipts DataFrame (by receipt_id):")
print(receipts_df.duplicated(subset=['_id']).sum())

print("Duplicate records in Brands DataFrame (by brand_id):")
print(brands_df.duplicated(subset=['_id']).sum())


Duplicate records in Users DataFrame (by user_id):
283
Duplicate records in Receipts DataFrame (by receipt_id):
0
Duplicate records in Brands DataFrame (by brand_id):
0


## Investigating data types

In [13]:
# Check for data types and fix them if necessary
print("Receipts DataFrame data types:")
print(receipts_df.dtypes)

# Convert total_spent to numeric if it's not already
receipts_df['totalSpent'] = pd.to_numeric(receipts_df['totalSpent'], errors='coerce')


Receipts DataFrame data types:
_id                         object
bonusPointsEarned          float64
bonusPointsEarnedReason     object
createDate                  object
dateScanned                 object
finishedDate                object
modifyDate                  object
pointsAwardedDate           object
pointsEarned               float64
purchaseDate                object
purchasedItemCount         float64
rewardsReceiptItemList      object
rewardsReceiptStatus        object
totalSpent                 float64
userId                      object
dtype: object


In [21]:
def extract_date(json_obj):
    try:
        return pd.to_datetime(json_obj['$date'], unit='ms')
    except (TypeError, KeyError):
        return pd.NaT

# Apply this function to all relevant date columns
receipts_df['createDate'] = receipts_df['createDate'].apply(extract_date)
receipts_df['dateScanned'] = receipts_df['dateScanned'].apply(extract_date)
receipts_df['finishedDate'] = receipts_df['finishedDate'].apply(extract_date)
receipts_df['modifyDate'] = receipts_df['modifyDate'].apply(extract_date)
receipts_df['pointsAwardedDate'] = receipts_df['pointsAwardedDate'].apply(extract_date)
receipts_df['purchaseDate'] = receipts_df['purchaseDate'].apply(extract_date)

# Verify the changes
receipts_df.dtypes

_id                                object
bonusPointsEarned                 float64
bonusPointsEarnedReason            object
createDate                 datetime64[ns]
dateScanned                datetime64[ns]
finishedDate               datetime64[ns]
modifyDate                 datetime64[ns]
pointsAwardedDate          datetime64[ns]
pointsEarned                      float64
purchaseDate               datetime64[ns]
purchasedItemCount                float64
rewardsReceiptItemList             object
rewardsReceiptStatus               object
totalSpent                        float64
userId                             object
purchase_date              datetime64[ns]
dtype: object

In [22]:
# Convert the date fields in users data to datetime
users_df['createdDate'] = users_df['createdDate'].apply(extract_date)
users_df['lastLogin'] = users_df['lastLogin'].apply(extract_date)

# Verify the changes
users_df.dtypes

_id                     object
active                    bool
createdDate     datetime64[ns]
lastLogin       datetime64[ns]
role                    object
signUpSource            object
state                   object
dtype: object

In [24]:
# Checking data types of the other two tables (Brands and Users)
brands_data_types = brands_df.dtypes
users_data_types = users_df.dtypes

brands_data_types, users_data_types

(_id              object
 barcode           int64
 category         object
 categoryCode     object
 cpg              object
 name             object
 topBrand        float64
 brandCode        object
 dtype: object,
 _id                     object
 active                    bool
 createdDate     datetime64[ns]
 lastLogin       datetime64[ns]
 role                    object
 signUpSource            object
 state                   object
 dtype: object)

## Investigating receipts with total spend = 0

In [16]:
# Check for negative or suspicious values in total_spent or points_earned
print("Receipts with negative or zero total_spent:")
print(receipts_df[receipts_df['totalSpent'] <= 0])



Receipts with negative or zero total_spent:
                                      _id  bonusPointsEarned  \
15   {'$oid': '5ff1e1e90a7214ada1000569'}                NaN   
81   {'$oid': '5ff4ce3c0a720f05230005c4'}                NaN   
141  {'$oid': '5ff73be90a720f052300060a'}                NaN   
144  {'$oid': '5ff794600a7214ada1000647'}                NaN   
175  {'$oid': '5ff8da570a720f05c5000015'}                NaN   
179  {'$oid': '5ff8da7d0a720f05c500001c'}                NaN   
282  {'$oid': '6000d4aa0a720f05f3000072'}                NaN   
396  {'$oid': '6009eb000a7214ada2000003'}              250.0   
424  {'$oid': '600aff160a720f053500000c'}              500.0   
544  {'$oid': '60132b890a7214ad50000013'}                NaN   
559  {'$oid': '60145a540a720f05f8000116'}                NaN   
642  {'$oid': '601830cd0a720f05f800034f'}                NaN   
656  {'$oid': '60182f390a7214ad500002b8'}                NaN   
681  {'$oid': '60189c920a7214ad2800003a'}                NaN

In [18]:
# Investigating receipts marked as "FINISHED" but with a totalSpent of 0

# Filter for receipts that are "FINISHED" but have a totalSpent of 0
finished_zero_spent_receipts = receipts_df[(receipts_df['rewardsReceiptStatus'] == 'FINISHED') & (receipts_df['totalSpent'] == 0)]

# Checking the number of these receipts and examining their key fields
finished_zero_spent_count = finished_zero_spent_receipts.shape[0]
finished_zero_spent_sample = finished_zero_spent_receipts[['userId', 'purchaseDate', 'pointsEarned', 'totalSpent', 'rewardsReceiptItemList']]

# Check for potential patterns, e.g., presence of 'needsFetchReview' or unusual item descriptions in rewardsReceiptItemList
needs_fetch_review_count = finished_zero_spent_receipts['rewardsReceiptItemList'].apply(lambda x: any(item.get('needsFetchReview', False) for item in x) if isinstance(x, list) else False).sum()

finished_zero_spent_count, needs_fetch_review_count, finished_zero_spent_sample.head()


(2,
 0,
                        userId              purchaseDate  pointsEarned  \
 396  6009e60450b3311194385009  {'$date': 1611187200000}         250.0   
 424  600afb2a7d983a124e9aded0  {'$date': 1611273600000}         500.0   
 
      totalSpent rewardsReceiptItemList  
 396         0.0                    NaN  
 424         0.0                    NaN  )

## Investigating purchase_date to see if there is any date from the future added

In [20]:
from datetime import datetime
import pandas as pd

# Function to extract '$date' from dict and convert to datetime
def convert_to_datetime(date_field):
    if isinstance(date_field, dict) and '$date' in date_field:
        # Convert timestamp (milliseconds) to datetime
        return pd.to_datetime(date_field['$date'], unit='ms')
    return pd.NaT  # Return 'Not a Time' if invalid

# Apply the conversion to the purchaseDate field
receipts_df['purchase_date'] = receipts_df['purchaseDate'].apply(convert_to_datetime)

# Now check for future dates
print("Receipts with purchase_date in the future:")
future_receipts = receipts_df[receipts_df['purchase_date'] > datetime.now()]
print(future_receipts)

Receipts with purchase_date in the future:
Empty DataFrame
Columns: [_id, bonusPointsEarned, bonusPointsEarnedReason, createDate, dateScanned, finishedDate, modifyDate, pointsAwardedDate, pointsEarned, purchaseDate, purchasedItemCount, rewardsReceiptItemList, rewardsReceiptStatus, totalSpent, userId, purchase_date]
Index: []


So all dates are completely fine since an empty data frame was returned

## Checking for Unique and consistent values

In [25]:
brands_category_unique = brands_df['category'].unique()
users_state_unique = users_df['state'].unique()
users_role_unique = users_df['role'].unique()
users_signUpSource_unique = users_df['signUpSource'].unique()

brands_category_unique, users_state_unique, users_role_unique, users_signUpSource_unique


(array(['Baking', 'Beverages', 'Candy & Sweets', 'Condiments & Sauces',
        'Canned Goods & Soups', nan, 'Magazines', 'Breakfast & Cereal',
        'Beer Wine Spirits', 'Health & Wellness', 'Beauty', 'Baby',
        'Frozen', 'Grocery', 'Snacks', 'Household', 'Personal Care',
        'Dairy', 'Cleaning & Home Improvement', 'Deli',
        'Beauty & Personal Care', 'Bread & Bakery', 'Outdoor',
        'Dairy & Refrigerated'], dtype=object),
 array(['WI', 'KY', 'AL', 'CO', 'IL', nan, 'OH', 'SC', 'NH'], dtype=object),
 array(['consumer', 'fetch-staff'], dtype=object),
 array(['Email', 'Google', nan], dtype=object))

## Checking for Outliers


In [26]:
# For Brands topBrand
brands_topBrand_outliers = brands_df['topBrand'].describe()

# For Users (days since last login)
users_df['days_since_login'] = (pd.Timestamp.now() - users_df['lastLogin']).dt.days
users_lastLogin_outliers = users_df['days_since_login'].describe()

brands_topBrand_outliers, users_lastLogin_outliers


(count    555.000000
 mean       0.055856
 std        0.229850
 min        0.000000
 25%        0.000000
 50%        0.000000
 75%        0.000000
 max        1.000000
 Name: topBrand, dtype: float64,
 count     433.000000
 mean     1359.270208
 std        51.310858
 min      1318.000000
 25%      1348.000000
 50%      1361.000000
 75%      1374.000000
 max      2351.000000
 Name: days_since_login, dtype: float64)

In [27]:
total_spent_outliers = receipts_df['totalSpent'].describe()
total_spent_outliers

count     684.000000
mean       77.796857
std       347.110349
min         0.000000
25%         1.000000
50%        18.200000
75%        34.960000
max      4721.950000
Name: totalSpent, dtype: float64

In [29]:
points_earned_outliers = receipts_df['pointsEarned'].describe()
points_earned_outliers

count      609.000000
mean       585.962890
std       1357.166947
min          0.000000
25%          5.000000
50%        150.000000
75%        750.000000
max      10199.800000
Name: pointsEarned, dtype: float64

In [30]:
purchased_item_count_outliers = receipts_df['purchasedItemCount'].describe()
purchased_item_count_outliers

count    635.00000
mean      14.75748
std       61.13424
min        0.00000
25%        1.00000
50%        2.00000
75%        5.00000
max      689.00000
Name: purchasedItemCount, dtype: float64